# Inferential Statistics : Capstone Project 1 

### Objective: Apply Inferential statistics techniques to explore the data

#### Step 1: Read CSV file with Airbnb data downloaded from http://insideairbnb.com/get-the-data.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%pylab inline
df = pd.read_csv('/Users/doyelm/Documents/Project/listings.csv')



Populating the interactive namespace from numpy and matplotlib


#### Step2: Study the data

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9663 entries, 0 to 9662
Data columns (total 16 columns):
id                                9663 non-null int64
name                              9663 non-null object
host_id                           9663 non-null int64
host_name                         9661 non-null object
neighbourhood_group               0 non-null float64
neighbourhood                     9663 non-null int64
latitude                          9663 non-null float64
longitude                         9663 non-null float64
room_type                         9663 non-null object
price                             9663 non-null int64
minimum_nights                    9663 non-null int64
number_of_reviews                 9663 non-null int64
last_review                       6006 non-null object
reviews_per_month                 6007 non-null float64
calculated_host_listings_count    9663 non-null int64
availability_365                  9663 non-null int64
dtypes: float64(4), 

In [8]:
df2 = df.groupby(['neighbourhood'])[['id']].count().reset_index()
 
len(df2[df2.id > 10].sort_values('id',ascending=False ) )

41

#### Select top 39 neighbourhoods - from above result we see 39 neighbourhoods have sample > 10

In [3]:

df_1=df.groupby(['neighbourhood' ])[['id']].count().sort_values(by=['id'], ascending=False)[0:39]

# choose data for the neighbourhoods in the list created above
df_new = df[df['neighbourhood'].isin(df_1.index.values.tolist())].loc[:, ('neighbourhood', 'latitude','longitude','room_type','price','minimum_nights','number_of_reviews', 'availability_365') ]
df_new.fillna(0)   
df_new.price=df_new.price.astype(float) 
df_new.minimum_nights=df_new.minimum_nights.astype(float) 
df_new.number_of_reviews=df_new.number_of_reviews.astype(float) 
df_new.availability_365=df_new.availability_365.astype(float) 
df_new.neighbourhood=df_new.neighbourhood.astype(str) 
 
df_new.info()
 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9621 entries, 0 to 9662
Data columns (total 8 columns):
neighbourhood        9621 non-null object
latitude             9621 non-null float64
longitude            9621 non-null float64
room_type            9621 non-null object
price                9621 non-null float64
minimum_nights       9621 non-null float64
number_of_reviews    9621 non-null float64
availability_365     9621 non-null float64
dtypes: float64(6), object(2)
memory usage: 676.5+ KB


### Check if data exists with price == 0

In [127]:
df[df.price == 0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
last_review,,,,,,,,,,,,,,,,


### convert non-numeric binary variables into binary (0/1) variables. Converts categorical variables into dummy variables.

In [8]:

def preprocess_features(X):
    '''   converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''

    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)
    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        #print(col,col_data)
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

#y =  log(df_new.price+1)#.fillna(0)   
y =  df_new.price
X = df_new.loc[:,df_new.columns != 'price'] 
X_all = preprocess_features(X)
 
 

 

## Apply Inferential Statictics techniques

### Split the targets into training/testing sets

In [6]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(X_all, y, test_size=0.3, random_state=42)

### Perform T test  or Hypothesis testing

In [9]:
from scipy import stats
 
df_2=df.groupby(['room_type' ])[['id']].count().sort_values(by=['id'] , ascending=False)[0:1]
df_3=df.groupby(['room_type'])[['id']].count().sort_values(by=['id'] , ascending=False)[1:2]
#print(df_2)
#print(df_3)
df_array2=df[df['room_type'].isin(df_2.index.values.tolist())].price
df_array3=df[df['room_type'].isin(df_3.index.values.tolist())].price

#print(df_array1)
np.seterr(divide='ignore', invalid='ignore')
stats.ttest_ind(df_array2, df_array3, equal_var = False)

Ttest_indResult(statistic=32.78944417358676, pvalue=3.181358246813975e-222)

<P> In the above test stats.ttest_ind is used to calculate the T-test for the means of two independent samples of scores. 
    The test measures whether the average (expected) value differs significantly across samples. 
    If the P-value is less than the significance level 0.05, we reject the null hypothesis.<p>
    <p><b>Null hypothesis :<b> "The average price for the apartment is not equal to the average price for the private room" <p>
    <p><b>The alternatve hypothesis :<b> " the average price is equal for different types of rooms. <p>
   <p>As P-value is larger than 0.05 or 0.1, we reject the null hypothesis 
<p>



### OLS - Ordinary Least squares model 

In [10]:

import statsmodels
from numpy.random import randn
import statsmodels.api as sm

%pylab inline

mymodel = sm.OLS(y,X_all)
results = mymodel.fit()
results.params
results.summary()
#results.summary2()
results_summary=results.summary2()
print(results_summary)



Populating the interactive namespace from numpy and matplotlib
                             Results: Ordinary least squares
Model:                      OLS                     Adj. R-squared:            0.140      
Dependent Variable:         price                   AIC:                       143195.8225
Date:                       2018-05-15 22:12        BIC:                       143525.7209
No. Observations:           9621                    Log-Likelihood:            -71552.    
Df Model:                   45                      F-statistic:               35.68      
Df Residuals:               9575                    Prob (F-statistic):        2.19e-282  
R-squared:                  0.144                   Scale:                     1.6957e+05 
------------------------------------------------------------------------------------------
                             Coef.      Std.Err.     t     P>|t|     [0.025       0.975]  
---------------------------------------------------------

#### The OLS class estimates a multi-variate regression model and provides a variety of fit-statistics. 
#### The negative values for the constants simply means that the expected value on the dependent variable (price) will be less than 0 when all independent/predictor variables are set to 0. 
#### Looking into the coefficients we see that room type is a significant factor followed by neighbourhood in determining the price of rooms.

### linear regression - mean squared deviation - for neighbourhood and price

In [12]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


# Create linear regression object
regr = linear_model.LinearRegression()


# Train the model using the training sets
regr.fit(df_X_train,df_y_train)

# Make predictions using the testing set
df_y_pred = regr.predict(df_X_test)


from sklearn.metrics import r2_score

print(' R-squared:',r2_score(df_y_test, df_y_pred)) 



 R-squared: 0.4312684928007976
